In [6]:
#Script Name: MolProps_with_RDKIT
#Script Purpose: Calculating RDkit molecular properties. Requires RDkit in your environment
#Author Name: John Bickel
#Affiliation: Rizzo Lab, Stony Brook University
#Create date: 03/04/2021 (MM/DD/YYYY)
#Last edit: 03/04/2021 John Bickel/SBU


# Import modules for our exercise
import numpy as np     # numerical package
import pandas as pd    # data organization package
import matplotlib.pyplot as plt # plotting package
import os              # operational system interface package
%matplotlib inline

In [7]:
# Import modules for dealing with chemical information
from rdkit import Chem as Chem
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import AllChem as Chem2
from rdkit.Chem import Descriptors as Desc
from rdkit.Chem import Draw
from rdkit.Chem import rdmolfiles as RDFile

import pickle

import math
from collections import defaultdict

import os.path as op

In [8]:
# Function that reads multi-molecule MOL2 files. Adapted from:
# https://chem-workflows.com/articles/2019/07/18/building-a-multi-molecule-mol2-reader-for-rdkit/
# further adapted from function written by Guilherme Duarte, Rizzo Lab
def mol2_mol_supplier_loop(file):
    ''' This function extracts all the molecules in the multi-molecule
        MOL2 file `file` and returns a list of rdkit.Chem.rdchem.mol 
        object.
        
        Variable         I/O          dtype           default value?
        ------------------------------------------------------------
        file              I           string                  None
        mols              O           list                    N/A
        mols[i]           O           rdkit.Chem.rdchem.mol   N/A
        
    '''
    mols=[]
    
    recording=False
    with open(file, 'r') as f:
        for line in f:
            
            # Determines if @<TRIPOS>MOLECULE is in line, which marks the start
            # of each molecule. Sets the recording variable to True, which is
            # the boolean to write each molecule.
            if ("@<TRIPOS>MOLECULE") in line:
                recording=True
                mol = []
            # Determines if "ROOT" is in line, which marks the end of each
            # molecule. Records the line and sets the recording variable
            # to false.
            elif ("ROOT") in line:
                mol.append(line)
                recording=False
                
                # Makes final adjustments to the data. It must look
                # like the MOL2 file of a single molecule.
                block = ",".join(mol).replace(',','')
                
                # Converts the data of a single molecule to a 
                # rdkit.Chem.rdchem.mol object.
                m=Chem.MolFromMol2Block(block,
                                        sanitize=False,
                                        cleanupSubstructures=False)
                mols.append(m)
                continue
                
            if recording==True:
                mol.append(line)
                
        return(mols)
                

# Don't use this unless you need to break things up

In [9]:
#### A breakup step if you need to truncate the # of molecules you're working with
#### for them to be workable with the amount of available memory.
filename="./2020.01.03_roulette_fixes_combined_molecules.mol2"
o_filename="test_out.mol2"

molecule_counter=0
with open(filename,'r') as f:
    with open(o_filename,'w') as o:
        for line in f:
            if "Name" in line:
                molecule_counter+=1
            if molecule_counter > 100:
                break
            if "###" in line:
                continue
            o.write(line)

# Actual Code

In [ ]:
filename="./2020.01.03_roulette_fixes_combined_molecules.mol2"

molecule_list=mol2_mol_supplier_loop(filename)

In [ ]:
#This is a sanity check step - to make sure everything worked above.
for mol in molecule_list:
    if mol is None: continue
    print(mol.GetNumAtoms())

In [ ]:
for mol in molecule_list:
    mol.UpdatePropertyCache(strict=False)
    RDFile.MolToPDBFile(mol,"out1.pdb")
    Chem.SanitizeMol(mol,sanitizeOps=Chem.SanitizeFlags.SANITIZE_ALL^Chem.SanitizeFlags.SANITIZE_KEKULIZE^Chem.SanitizeFlags.SANITIZE_SETAROMATICITY)
    RDFile.MolToPDBFile(mol,"out2.pdb")

## Aromatic Rings

In [ ]:
aromatic_count_list=[]
for mol in molecule_list:
    aromatic_count_list.append(Desc.NumAromaticRings(mol))

In [ ]:
fig=plt.figure(num=None, figsize=(8, 2), dpi=400, facecolor='w', edgecolor='k',)
plt.title("NumAromaticRings")
plt.hist(aromatic_count_list,density=True,bins=[0,1,2,3,4,5,6])

In [ ]:
with open("aromatic_ring_list.dat","w") as f:
    for i in aromatic_count_list:
        f.write("%s\n" % (i))
aromatic_count_list.clear()

## LogP

In [ ]:
logp_list=[]
for mol in molecule_list:
    logp_list.append(Chem.Crippen.MolLogP(mol))

In [ ]:
fig=plt.figure(num=None, figsize=(8, 2), dpi=400, facecolor='w', edgecolor='k',)
plt.title("LogP")
plt.hist(logp_list,bins=50,density=True)

In [ ]:
with open("logp.dat","w") as f:
    for i in logp_list:

        f.write("%s\n" % (i))
logp_list.clear()

## Molecular Weight

In [ ]:
mw_list=[]
for mol in molecule_list:
    mw_list.append(Desc.ExactMolWt(mol))

In [ ]:
with open("mw_list.dat","w") as f:
    for i in mw_list:

        f.write("%s\n" % (i))

In [ ]:
fig=plt.figure(num=None, figsize=(8, 2), dpi=400, facecolor='w', edgecolor='k',)
plt.title("MolecularWeight")
plt.hist(mw_list,bins=50,density=True)

# Others

In [ ]:
numhacceptors

In [ ]:
numhdonors

In [ ]:
#Hacc
bad_acc_counter=0
mol_counter=0
num_h_acc_list=[]
for mol in molecule_list:
    mol_counter+=1
    try:
        mol.UpdatePropertyCache(strict=False)
        num_h_acc_list.append(Desc.NumHAcceptors(mol))
        
    except:
        bad_acc_counter+=1
        
        continue

with open("numhacc_list.dat","w") as f:
    for i in num_h_acc_list:

        f.write("%s\n" % (i))
num_h_acc_list.clear() 
        
#Hdon
num_h_don_list=[]
bad_don_counter=0
mol_counter=0
for mol in molecule_list:
    mol_counter+=1
    try:
        mol.UpdatePropertyCache(strict=False)
        num_h_don_list.append(Desc.NumHDonors(mol))
        #print(Desc.NumHDonors(mol))
        
    except:
        bad_don_counter+=1
        
        continue

with open("numhdon_list.dat","w") as f:
    for i in num_h_don_list:

        f.write("%s\n" % (i))
num_h_acc_list.clear()
print("Bad acc: %d" % (bad_acc_counter))
print("Bad don: %d" % (bad_don_counter))

In [39]:
_fscores = None


def readFragmentScores(name='fpscores'):
    import gzip
    global _fscores
    # generate the full path filename:
    if name == "fpscores":
        print(os.getcwd())
        name = op.join(os.getcwd(), name)
    data = pickle.load(gzip.open('%s.pkl.gz' % name))
    outDict = {}
    for i in data:
        for j in range(1, len(i)):
            outDict[i[j]] = float(i[0])
    _fscores = outDict


def numBridgeheadsAndSpiro(mol, ri=None):
    nSpiro = rdMolDescriptors.CalcNumSpiroAtoms(mol)
    nBridgehead = rdMolDescriptors.CalcNumBridgeheadAtoms(mol)
    return nBridgehead, nSpiro


def calculateScore(m):
    if _fscores is None:
        readFragmentScores()
    m.UpdatePropertyCache(strict=False)
    # fragment score
    fp = rdMolDescriptors.GetMorganFingerprint(m,
                                               2)  # <- 2 is the *radius* of the circular fingerprint
    fps = fp.GetNonzeroElements()
    score1 = 0.
    nf = 0
    for bitId, v in fps.items():
        nf += v
        sfp = bitId
        score1 += _fscores.get(sfp, -4) * v
    score1 /= nf

    # features score
    nAtoms = m.GetNumAtoms()
    nChiralCenters = len(Chem.FindMolChiralCenters(m, includeUnassigned=True))
    ri = m.GetRingInfo()
    nBridgeheads, nSpiro = numBridgeheadsAndSpiro(m, ri)
    nMacrocycles = 0
    for x in ri.AtomRings():
        if len(x) > 8:
            nMacrocycles += 1

    sizePenalty = nAtoms**1.005 - nAtoms
    stereoPenalty = math.log10(nChiralCenters + 1)
    spiroPenalty = math.log10(nSpiro + 1)
    bridgePenalty = math.log10(nBridgeheads + 1)
    macrocyclePenalty = 0.
    # ---------------------------------------
    # This differs from the paper, which defines:
    #  macrocyclePenalty = math.log10(nMacrocycles+1)
    # This form generates better results when 2 or more macrocycles are present
    if nMacrocycles > 0:
        macrocyclePenalty = math.log10(2)

    score2 = 0. - sizePenalty - stereoPenalty - spiroPenalty - bridgePenalty - macrocyclePenalty

    # correction for the fingerprint density
    # not in the original publication, added in version 1.1
    # to make highly symmetrical molecules easier to synthetise
    score3 = 0.
    if nAtoms > len(fps):
        score3 = math.log(float(nAtoms) / len(fps)) * .5

    sascore = score1 + score2 + score3

    # need to transform "raw" value into scale between 1 and 10
    min = -4.0
    max = 2.5
    sascore = 11. - (sascore - min + 1) / (max - min) * 9.
    # smooth the 10-end
    if sascore > 8.:
        sascore = 8. + math.log(sascore + 1. - 9.)
    if sascore > 10.:
        sascore = 10.0
    elif sascore < 1.:
        sascore = 1.0
    return sascore


In [ ]:
sasa_list=[]

for m in molecule_list:
    sasa_list.append(calculateScore(m))
    
with open("num_list.dat","w") as f:
    for i in sasa_list:
        f.write("%s\n" % (i))
        
sasa_list.clear() 